In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

## Part 1: Scrape Coronavirus Mainland China Data from wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/Timeline_of_the_2019%E2%80%9320_Wuhan_coronavirus_outbreak'

In [3]:
tables = pd.read_html(url)
# Since the 1st table on Wiki includes three tables, the new_confirmed table is the 4th.

### 1. New Confirmed Cases of Coronavirus in Mainland China by Provincial Divisions

In [4]:
# Since the 1st table on Wiki includes three tables, the new_confirmed table is the 4th.
# Check the infomation on in the Dataframe
df_China_new_confirmed_original = tables[3]
df_China_new_confirmed_original.info()
df_China_new_confirmed_original.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 34 columns):
Date (CST)        24 non-null object
Hubei             20 non-null object
Guangdong         16 non-null object
Beijing           15 non-null object
Shanghai          15 non-null object
Zhejiang          14 non-null object
Tianjin           14 non-null object
Chongqing         14 non-null object
Jiangxi           14 non-null object
Shandong          14 non-null object
Henan             14 non-null object
Hunan             14 non-null object
Sichuan           14 non-null object
Yunnan            13 non-null object
Shanxi            12 non-null object
Fujian            13 non-null object
Liaoning          13 non-null object
Hainan            13 non-null object
Anhui             13 non-null object
Guizhou           13 non-null object
Guangxi           13 non-null object
Ningxia           13 non-null object
Hebei             13 non-null object
Jiangsu           13 non-null object
Jilin    

,Date (CST),Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet,National,ExcludingHubei
0,2020-01-11,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,NaN
1,2020-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# remove the last column of the dataframe
df_China_new_confirmed_c = df_China_new_confirmed_original.iloc[:,:-2]
# remove the last 2 rows of the dataframe
df_China_new_confirmed_r = df_China_new_confirmed_c[:-2]
# Check the tail to see if the row&column removed
df_China_new_confirmed_r.tail()

,Date (CST),Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
17,2020-01-28,840,53,11,14,123,1,15,37,34,...,15,29,1,7,10,3,5,2,0,NaN
18,2020-01-29,1032,70,20,21,132,3,18,53,24,...,17,30,5,6,7,1,2,3,0,1
19,2020-01-30,1220,82,21,27,109,5,41,78,33,...,17,39,0,16,24,3,3,2,2,0
20,2020-01-31,1347,142,24,41,62,5,41,46,28,...,14,34,3,21,14,1,6,3,1,0
21,2020-02-01,1921,69,27,8,62,8,15,47,19,...,8,34,6,15,15,2,5,4,2,0


In [6]:
# # Rename the 1st column 
df_China_new_confirmed_r = df_China_new_confirmed_r.rename(columns={"Date (CST)" : "Date"})
# df_China_new_confirmed_r['Date']=df_China_new_confirmed_r['Date'].str.replace(".", "-")

df_China_new_confirmed = df_China_new_confirmed_r.copy()
# convert 'Date' column format to datetime
df_China_new_confirmed['Date'] = pd.to_datetime(df_China_new_confirmed['Date'], format= "%Y-%m-%d")
# df_China_new_confirmed.info()
# df_China_new_confirmed.head()

In [7]:
# Reset the index as the first column, this is for future use cumsum function
df_China_new_confirmed = df_China_new_confirmed.set_index(df_China_new_confirmed.columns[0])
# df_China_new_confirmed.head()

In [8]:
# fill N/A by 0
df_China_new_confirmed=df_China_new_confirmed.fillna(0)
# df_China_new_confirmed

In [9]:
df_China_new_confirmed_final = df_China_new_confirmed.astype(int) 
# df_China_new_confirmed_final.info()
# df_China_new_confirmed_final.tail()

* __The China Accumulated Confirmed Cases Table__ (Database)

In [10]:
# Sum the daliy confirmed patient
df_China_confirmed_final = df_China_new_confirmed_final.cumsum()
# Since the cumsum() function only works with integer then we change the index back as column
df_China_confirmed_final=df_China_confirmed_final.reset_index()
df_China_confirmed_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
17,2020-01-28,3554,241,91,80,296,24,147,109,121,...,48,99,9,37,56,13,24,15,6,0
18,2020-01-29,4586,311,111,101,428,27,165,162,145,...,65,129,14,43,63,14,26,18,6,1
19,2020-01-30,5806,393,132,128,537,32,206,240,178,...,82,168,14,59,87,17,29,20,8,1
20,2020-01-31,7153,535,156,169,599,37,247,286,206,...,96,202,17,80,101,18,35,23,9,1
21,2020-02-01,9074,604,183,177,661,45,262,333,225,...,104,236,23,95,116,20,40,27,11,1


* __The China New Confirmed Cases Table__ (Database)

In [11]:
df_China_new_confirmed_final=df_China_new_confirmed_final.reset_index()
df_China_new_confirmed_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
17,2020-01-28,840,53,11,14,123,1,15,37,34,...,15,29,1,7,10,3,5,2,0,0
18,2020-01-29,1032,70,20,21,132,3,18,53,24,...,17,30,5,6,7,1,2,3,0,1
19,2020-01-30,1220,82,21,27,109,5,41,78,33,...,17,39,0,16,24,3,3,2,2,0
20,2020-01-31,1347,142,24,41,62,5,41,46,28,...,14,34,3,21,14,1,6,3,1,0
21,2020-02-01,1921,69,27,8,62,8,15,47,19,...,8,34,6,15,15,2,5,4,2,0


### 2. New deaths from coronavirus in mainland China by provincial divisions
##### Repeat the same step as above

In [12]:
# Table of new death
df_China_new_deathes_original = tables[4]
df_China_new_deathes_original.tail()

,Date (CST),Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet,National
18,2020-01-29,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38[39]
19,2020-01-30,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,43[27]
20,2020-01-31,45.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46[38]
21,2020-02-01,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45[40]
22,Net,294.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,304


In [13]:
# remove the last column of the dataframe
df_China_new_deathes_c = df_China_new_deathes_original.iloc[:,:-1]
# remove the last 2 rows of the dataframe
df_China_new_deathes_r = df_China_new_deathes_c[:-1]
# df_China_new_deathes_r.tail()

In [14]:
# # Rename the 1st column 
df_China_new_deathes_r = df_China_new_deathes_r.rename(columns={"Date (CST)" : "Date"})
# df_China_new_confirmed_r['Date']=df_China_new_confirmed_r['Date'].str.replace(".", "-")

df_China_new_deathes = df_China_new_deathes_r.copy()
# convert 'Date' column format to datetime
df_China_new_deathes['Date'] = pd.to_datetime(df_China_new_deathes['Date'], format= "%Y-%m-%d")

In [15]:
# Reset the index as the first column, this is for future use cumsum function
df_China_new_deathes = df_China_new_deathes.set_index(df_China_new_deathes.columns[0])

In [16]:
# fill N/A by 0
df_China_new_deathes=df_China_new_deathes.fillna(0)

In [17]:
df_China_new_deathes_final = df_China_new_deathes.astype(int) 

* __The China Accumulated Deathes Cases Table__ (Database)

In [18]:
# Sum the daliy confirmed patient
df_China_deathes_final = df_China_new_deathes_final.cumsum()
# Since the cumsum() function only works with integer then we change the index back as column
df_China_deathes_final=df_China_deathes_final.reset_index()
df_China_deathes_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
17,2020-01-28,125,0,1,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
18,2020-01-29,162,0,1,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
19,2020-01-30,204,0,1,1,0,0,0,0,0,...,1,0,0,2,0,0,0,0,0,0
20,2020-01-31,249,0,1,1,0,0,0,1,0,...,1,0,0,2,0,0,0,0,0,0
21,2020-02-01,294,0,1,1,0,0,0,1,0,...,1,0,0,2,0,0,0,0,0,0


* __The China New Deathes Cases Table__ (Database)

In [19]:
df_China_new_deathes_final=df_China_new_deathes_final.reset_index()
df_China_new_deathes_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
17,2020-01-28,25,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,2020-01-29,37,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,2020-01-30,42,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
20,2020-01-31,45,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
21,2020-02-01,45,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3. New recovered cases of coronavirus in mainland China by provincial divisions
##### Repeat the same step as above

In [20]:
# Table of new recovered
df_China_new_recovered_original = tables[5]
df_China_new_recovered_original.tail()

,Date (CST),Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet,National
18,2020-01-29,10.0,1.0,NaN,1.0,1.0,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21[39]
19,2020-01-30,26.0,3.0,NaN,1.0,5.0,NaN,NaN,4.0,1.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47
20,2020-01-31,50.0,1.0,NaN,4.0,6.0,NaN,NaN,2.0,1.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72[38]
21,2020-02-01,49.0,2.0,4.0,1.0,8.0,NaN,2.0,1.0,2.0,...,1.0,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,85[40]
22,Net,215.0,12.0,9.0,10.0,23.0,NaN,3.0,10.0,5.0,...,6.0,1.0,2.0,NaN,NaN,NaN,1.0,NaN,NaN,328


In [21]:
# remove the last column of the dataframe
df_China_new_recovered_c = df_China_new_recovered_original.iloc[:,:-1]
# remove the last row of the dataframe
df_China_new_recovered_r = df_China_new_recovered_c[:-1]
# df_China_new_recovered_r.tail()

In [22]:
# # Rename the 1st column 
df_China_new_recovered_r = df_China_new_recovered_r.rename(columns={"Date (CST)" : "Date"})

df_China_new_recovered = df_China_new_recovered_r.copy()
# convert 'Date' column format to datetime
df_China_new_recovered['Date'] = pd.to_datetime(df_China_new_recovered['Date'], format= "%Y-%m-%d")

In [23]:
# Reset the index as the first column, this is for future use cumsum function
df_China_new_recovered = df_China_new_recovered.set_index(df_China_new_recovered.columns[0])

In [24]:
# fill N/A by 0
df_China_new_recovered=df_China_new_recovered.fillna(0)

In [25]:
df_China_new_recovered_final = df_China_new_recovered.astype(int) 

* __The China Accumulated Recovered Cases Table__ (Database)

In [26]:
# Sum the daliy recovered patient
df_China_recovered_final = df_China_new_recovered_final.cumsum()
# Since the cumsum() function only works with integer then we change the index back as column
df_China_recovered_final=df_China_recovered_final.reset_index()
df_China_recovered_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
17,2020-01-28,80,5,4,4,3,0,0,3,0,...,0,1,0,0,0,0,0,0,0,0
18,2020-01-29,90,6,4,5,4,0,1,3,1,...,0,1,0,0,0,0,0,0,0,0
19,2020-01-30,116,9,4,6,9,0,1,7,2,...,0,2,1,0,0,0,0,0,0,0
20,2020-01-31,166,10,4,10,15,0,1,9,3,...,0,5,1,0,0,0,0,0,0,0
21,2020-02-01,215,12,8,11,23,0,3,10,5,...,0,6,1,2,0,0,0,1,0,0


* __The China New Recovered Cases Table__ (Database)

In [27]:
df_China_new_recovered_final=df_China_new_recovered_final.reset_index()
df_China_new_recovered_final.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
17,2020-01-28,33,1,2,1,2,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
18,2020-01-29,10,1,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
19,2020-01-30,26,3,0,1,5,0,0,4,1,...,0,1,1,0,0,0,0,0,0,0
20,2020-01-31,50,1,0,4,6,0,0,2,1,...,0,3,0,0,0,0,0,0,0,0
21,2020-02-01,49,2,4,1,8,0,2,1,2,...,0,1,0,2,0,0,0,1,0,0


## Part 2: Import Coronavirus Mainland China Data Tables to SQLite

* __Table 1: The China Accumulated Confirmed Cases Table__ (Database)

### __Note : PK will add to sqlite by using "DB Broswer for SQLite"__

In [28]:
df_China_confirmed_final.columns

Index(['Date', 'Hubei', 'Guangdong', 'Beijing', 'Shanghai', 'Zhejiang',
       'Tianjin', 'Chongqing', 'Jiangxi', 'Shandong', 'Henan', 'Hunan',
       'Sichuan', 'Yunnan', 'Shanxi', 'Fujian', 'Liaoning', 'Hainan', 'Anhui',
       'Guizhou', 'Guangxi', 'Ningxia', 'Hebei', 'Jiangsu', 'Jilin',
       'Heilongjiang', 'Shaanxi', 'Xinjiang', 'Gansu', 'InnerMongolia',
       'Qinghai', 'Tibet'],
      dtype='object')

In [29]:
df_China_confirmed_final.head()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
0,2020-01-11,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-12,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-13,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-01-14,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-15,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# conn = sqlite3.connect('wuhan_pneumonia.sqlite')
# c = conn.cursor()
# c.execute("DROP TABLE IF EXISTS China_confirmed;")

In [ ]:
# c.execute('''
#           CREATE TABLE China_confirmed
#           ([Date] TEXT PRIMARY KEY, 
#            [Hubei] INTEGER NOT NULL,
#            [Guangdong] INTEGER NOT NULL,
#            [Beijing] INTEGER NOT NULL,
#            [Shanghai] INTEGER NOT NULL, 
#            [Zhejiang] INTEGER NOT NULL,
#            [Tianjin] INTEGER NOT NULL, 
#            [Chongqing] INTEGER NOT NULL, 
#            [Jiangxi] INTEGER NOT NULL, 
#            [Shandong] INTEGER NOT NULL, 
#            [Henan] INTEGER NOT NULL, 
#            [Hunan] INTEGER NOT NULL,
#            [Sichuan] INTEGER NOT NULL, 
#            [Yunnan] INTEGER NOT NULL, 
#            [Shanxi] INTEGER NOT NULL, 
#            [Fujian] INTEGER NOT NULL, 
#            [Liaoning] INTEGER NOT NULL, 
#            [Hainan] INTEGER NOT NULL, 
#            [Anhui] INTEGER NOT NULL,
#            [Guizhou] INTEGER NOT NULL, 
#            [Guangxi] INTEGER NOT NULL, 
#            [Ningxia] INTEGER NOT NULL, 
#            [Hebei] INTEGER NOT NULL, 
#            [Jiangsu] INTEGER NOT NULL, 
#            [Jilin] INTEGER NOT NULL,
#            [Heilongjiang] INTEGER NOT NULL, 
#            [Shaanxi] INTEGER NOT NULL, 
#            [Xinjiang] INTEGER NOT NULL, 
#            [Gansu] INTEGER NOT NULL, 
#            [InnerMongolia] INTEGER NOT NULL,
#            [Qinghai] INTEGER NOT NULL, 
#            [Tibet] INTEGER NOT NULL
#            )
#           ''')
# conn.commit()
# conn.close()

In [30]:
engine = create_engine('sqlite:///wuhan_pneumonia.sqlite')
df_China_confirmed_final.to_sql(name='China_confirmed', con=engine, index=False , if_exists='replace')

In [31]:
df_confirmed_China = pd.read_sql_query('SELECT * FROM China_confirmed',engine)
df_confirmed_China.tail()

,Date,Hubei,Guangdong,Beijing,Shanghai,Zhejiang,Tianjin,Chongqing,Jiangxi,Shandong,...,Hebei,Jiangsu,Jilin,Heilongjiang,Shaanxi,Xinjiang,Gansu,InnerMongolia,Qinghai,Tibet
17,2020-01-28 00:00:00.000000,3554,241,91,80,296,24,147,109,121,...,48,99,9,37,56,13,24,15,6,0
18,2020-01-29 00:00:00.000000,4586,311,111,101,428,27,165,162,145,...,65,129,14,43,63,14,26,18,6,1
19,2020-01-30 00:00:00.000000,5806,393,132,128,537,32,206,240,178,...,82,168,14,59,87,17,29,20,8,1
20,2020-01-31 00:00:00.000000,7153,535,156,169,599,37,247,286,206,...,96,202,17,80,101,18,35,23,9,1
21,2020-02-01 00:00:00.000000,9074,604,183,177,661,45,262,333,225,...,104,236,23,95,116,20,40,27,11,1


* __Table 2: The China New Confirmed Cases Table__ (Database)

In [ ]:
df_China_new_confirmed_final.to_sql('China_new_confirmed', engine, index=False,if_exists='replace')

In [ ]:
df_new_confirmed_China = pd.read_sql_query('SELECT * FROM China_new_confirmed',engine)
df_new_confirmed_China.tail()

* __Table 3: The China Accumulated Deathes Cases Table__ (Database)

In [ ]:
df_China_deathes_final.to_sql('China_deathes', engine, index=False,if_exists='replace')

In [ ]:
df_deathes_China = pd.read_sql_query('SELECT * FROM China_deathes',engine)
df_deathes_China.tail()

* __Table 4: The China New Deathes Cases Table__ (Database)

In [ ]:
df_China_new_deathes_final.to_sql('China_new_deathes', engine, index=False,if_exists='replace')

In [ ]:
df_new_deathes_China = pd.read_sql_query('SELECT * FROM China_new_deathes',engine)
df_new_deathes_China.tail()

* __Table 5: The China Accumulated Recovered Cases Table__ (Database)

In [ ]:
df_China_recovered_final.to_sql('China_recovered', engine, index=False,if_exists='replace')

In [ ]:
df_recovered_China = pd.read_sql_query('SELECT * FROM China_recovered',engine)
df_recovered_China.tail()

* __Table 6: The China New Recovered Cases Table__ (Database)

In [ ]:
df_China_new_recovered_final.to_sql('China_new_recovered', engine, index=False,if_exists='replace')

In [ ]:
df_new_recovered_China = pd.read_sql_query('SELECT * FROM China_new_recovered',engine)
df_new_recovered_China.tail()